<a href="https://colab.research.google.com/github/IshtiSikder/CNN-model-for-security-analysis-of-power-systems/blob/main/513_final_project_replicating_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [568]:
#Install gurobi
!pip install gurobipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [569]:
#Import necessary libraries
import sys
import numpy as np
import pandas as pd
from numpy import linalg as la
import gurobipy as gp
from gurobipy import *

In [570]:
#Activate gurobi license (only if run on google colab)
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', 'dcee8c55-6742-4816-bae1-a0742c8f71a9')
e.setParam('WLSSECRET', '70f89e9f-2426-41b9-abe7-88c4b6090a45')
e.setParam('LICENSEID',820825)
e.start()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 820825
Academic license - for non-commercial use only - registered to isikder@uncc.edu


<gurobipy.Env, Parameter changes: WLSAccessID=(user-defined), WLSSecret=(user-defined), LicenseID=820825>

In [571]:
#Setup the optimization model
New_Model = gp.Model("New Model",env=e) 

In [572]:
data = pd.DataFrame({'pre-demand':[100,150,0,0,0,0],'pre-demand prob':[0.5,0.5,0,0,0,0],'demand':[650,750,850,950,1050,1150],'demand prob':[0.17,0.27,0.23,0.17,0.12,0.04]})
data

,pre-demand,pre-demand prob,demand,demand prob
0,100,0.5,650,0.17
1,150,0.5,750,0.27
2,0,0.0,850,0.23
3,0,0.0,950,0.17
4,0,0.0,1050,0.12
5,0,0.0,1150,0.04


In [573]:
K = range(1,37)
T = [1,2]

In [574]:
Q = New_Model.addVars(K,T,lb=0,vtype=GRB.INTEGER)
v = New_Model.addVars(K,[0,1,2],lb=0,vtype=GRB.INTEGER)
u = New_Model.addVars(K,T,lb=0,vtype=GRB.INTEGER)
x = New_Model.addVars(K,T,lb=0,vtype=GRB.INTEGER)
q = New_Model.addVars(K,T,lb=0,vtype=GRB.INTEGER)
c = New_Model.addVars(T,lb=0,vtype=GRB.INTEGER)
h = New_Model.addVars(T,lb=0,vtype=GRB.INTEGER)
s = New_Model.addVars(T,lb=0,vtype=GRB.INTEGER)

In [575]:
for k in K:
  for t in T:
      if 1<=k<=6:
        x[k,t]=data['demand'][0] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
        q[k,t]=data['demand prob'][0]
      elif 7<=k<=12:
        x[k,t]=data['demand'][1] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
        q[k,t]=data['demand prob'][1]
      elif 13<=k<=18:
        x[k,t]=data['demand'][2] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
        q[k,t]=data['demand prob'][2]
      elif 19<=k<=24:
        x[k,t]=data['demand'][3] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
        q[k,t]=data['demand prob'][3]
      elif 25<=k<=30:
        x[k,t]=data['demand'][4] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
        q[k,t]=data['demand prob'][4]
      else:
        x[k,t]=data['demand'][5] + np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
        q[k,t]=data['demand prob'][5]
'''  
    else:
        x[k,t] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
        q[k,t] = data.loc[data.index[data['pre-demand']==x[k,t]].tolist()[0],'pre-demand prob']
'''

"  \n    else:\n        x[k,t] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]\n        q[k,t] = data.loc[data.index[data['pre-demand']==x[k,t]].tolist()[0],'pre-demand prob']\n"

In [576]:
Q_prime = New_Model.addVars([1,2],[0,1,2,3,4,5,6],lb=0,vtype=GRB.INTEGER)
u_prime = New_Model.addVars([2],[1,2,3,4,5,6],lb=0,vtype=GRB.INTEGER)
v_prime = New_Model.addVars([2],[1,2,3,4,5,6],lb=0,vtype=GRB.INTEGER)

In [577]:
#bin = New_Model.addVars(K,[0,1,2],vtype=GRB.BINARY)

In [578]:
c[1] = 30
c[2] = 4*c[1]
s[1] = 300
s[2] = s[1]

for t in T:
  h[t] = c[t]/2

In [579]:
'''
for k in K:
  for t in T:
    if t == 1:
      x[k,t] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]
      q[k,t] = data.loc[data.index[data['pre-demand']==x[k,t]].tolist()[0],'pre-demand prob']
    else:
      x[k,t] = np.random.choice(list(data['demand']), 1, list(data['demand prob']))[0]
      q[k,t] = data.loc[data.index[data['demand']==x[k,t]].tolist()[0],'demand prob']
'''

"\nfor k in K:\n  for t in T:\n    if t == 1:\n      x[k,t] = np.random.choice(list(data['pre-demand'])[:2], 1, list(data['pre-demand prob'])[:2])[0]\n      q[k,t] = data.loc[data.index[data['pre-demand']==x[k,t]].tolist()[0],'pre-demand prob']\n    else:\n      x[k,t] = np.random.choice(list(data['demand']), 1, list(data['demand prob']))[0]\n      q[k,t] = data.loc[data.index[data['demand']==x[k,t]].tolist()[0],'demand prob']\n"

In [580]:
New_Model.addConstrs(v[k,0]==0 for k in K)
#New_Model.addConstrs(bin[k,0]==0 for k in K)
#New_Model.addConstrs(bin[k,2]== bin[k,1]for k in K)

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>,
 22: <gurobi.Constr *Awaiting Model Update*

In [581]:
New_Model.addConstrs(Q[k,1] == Q_prime[1,0] for k in K)
New_Model.addConstrs(Q[k,2] == Q_prime[2,1] for k in K[:6])
New_Model.addConstrs(Q[k,2] == Q_prime[2,2] for k in K[6:12])
New_Model.addConstrs(Q[k,2] == Q_prime[2,3] for k in K[12:18])
New_Model.addConstrs(Q[k,2] == Q_prime[2,4] for k in K[18:24])
New_Model.addConstrs(Q[k,2] == Q_prime[2,5] for k in K[24:30])
New_Model.addConstrs(Q[k,2] == Q_prime[2,6] for k in K[30:36])

{31: <gurobi.Constr *Awaiting Model Update*>,
 32: <gurobi.Constr *Awaiting Model Update*>,
 33: <gurobi.Constr *Awaiting Model Update*>,
 34: <gurobi.Constr *Awaiting Model Update*>,
 35: <gurobi.Constr *Awaiting Model Update*>,
 36: <gurobi.Constr *Awaiting Model Update*>}

In [582]:
New_Model.addConstrs(u[k,2] == u_prime[2,1] for k in K[:6])
New_Model.addConstrs(u[k,2] == u_prime[2,2] for k in K[6:12])
New_Model.addConstrs(u[k,2] == u_prime[2,3] for k in K[12:18])
New_Model.addConstrs(u[k,2] == u_prime[2,4] for k in K[18:24])
New_Model.addConstrs(u[k,2] == u_prime[2,5] for k in K[24:30])
New_Model.addConstrs(u[k,2] == u_prime[2,6] for k in K[30:36])

{31: <gurobi.Constr *Awaiting Model Update*>,
 32: <gurobi.Constr *Awaiting Model Update*>,
 33: <gurobi.Constr *Awaiting Model Update*>,
 34: <gurobi.Constr *Awaiting Model Update*>,
 35: <gurobi.Constr *Awaiting Model Update*>,
 36: <gurobi.Constr *Awaiting Model Update*>}

In [583]:
New_Model.addConstrs(v[k,2] == v_prime[2,1] for k in K[:6])
New_Model.addConstrs(v[k,2] == v_prime[2,2] for k in K[6:12])
New_Model.addConstrs(v[k,2] == v_prime[2,3] for k in K[12:18])
New_Model.addConstrs(v[k,2] == v_prime[2,4] for k in K[18:24])
New_Model.addConstrs(v[k,2] == v_prime[2,5] for k in K[24:30])
New_Model.addConstrs(v[k,2] == v_prime[2,6] for k in K[30:36])

{31: <gurobi.Constr *Awaiting Model Update*>,
 32: <gurobi.Constr *Awaiting Model Update*>,
 33: <gurobi.Constr *Awaiting Model Update*>,
 34: <gurobi.Constr *Awaiting Model Update*>,
 35: <gurobi.Constr *Awaiting Model Update*>,
 36: <gurobi.Constr *Awaiting Model Update*>}

In [584]:
#New_Model.addConstrs(Q[k,t]+bin[k,t-1]*v[k,t-1]+(1-bin[k,t])*u[k,t]-bin[k,t]*v[k,t] == x[k,t] for k in K for t in T)

In [585]:
New_Model.addConstrs(Q[k,t]+v[k,t-1]+u[k,t]-v[k,t] == x[k,t] for k in K for t in T)

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (5, 1): <gurobi.Constr *Awaiting Model Update*>,
 (5, 2): <gurobi.Constr *Awaiting Model Update*>,
 (6, 1): <gurobi.Constr *Awaiting Model Update*>,
 (6, 2): <gurobi.Constr *Awaiting Model Update*>,
 (7, 1): <gurobi.Constr *Awaiting Model Update*>,
 (7, 2): <gurobi.Constr *Awaiting Model Update*>,
 (8, 1): <gurobi.Constr *Awaiting Model Update*>,
 (8, 2): <gurobi.Constr *Awaiting Model Update*>,
 (9, 1): <gurobi.Constr *Awaiting Model Update*>,
 (9, 2): <gurobi.Constr *Awaiting Model Update*>,
 (10, 1): <gurobi.Constr *Awaiting Model Update*>,
 (10, 2): <gurobi.Constr *Awaiting Model Update*>

In [586]:
obj = 0

for t in T:
  for k in K:
    obj += q[k,t]*(c[t]*Q[k,t] + h[t]*v[k,t] + s[t]*u[k,t])

New_Model.setObjective(obj, GRB.MINIMIZE)
New_Model.optimize()


print('Obj: %g' % New_Model.ObjVal)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license - for non-commercial use only - registered to isikder@uncc.edu
Optimize a model with 252 rows, 428 columns and 612 nonzeros
Model fingerprint: 0x0dddb7b7
Variable types: 0 continuous, 428 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e-01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+02, 1e+03]
Presolve removed 240 rows and 415 columns
Presolve time: 0.00s
Presolved: 12 rows, 13 columns, 24 nonzeros
Variable types: 0 continuous, 13 integer (0 binary)
Found heuristic solution: objective 1238163.9000

Root relaxation: objective 6.148050e+05, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf |

In [587]:
for i in [1,2]:
  for j in [0,1,2,3,4,5,6]:
    print(i,j,Q_prime[i,j].X)

1 0 1700.0
1 1 -0.0
1 2 -0.0
1 3 -0.0
1 4 -0.0
1 5 -0.0
1 6 -0.0
2 0 -0.0
2 1 0.0
2 2 0.0
2 3 200.0
2 4 400.0
2 5 650.0
2 6 850.0


In [588]:
for k in K:
  for t in T:
    print(k,t,x[k,t])

1 1 750
1 2 750
2 1 750
2 2 800
3 1 750
3 2 750
4 1 750
4 2 800
5 1 800
5 2 750
6 1 750
6 2 750
7 1 850
7 2 900
8 1 850
8 2 900
9 1 850
9 2 850
10 1 850
10 2 850
11 1 850
11 2 900
12 1 850
12 2 850
13 1 950
13 2 1000
14 1 1000
14 2 1000
15 1 1000
15 2 1000
16 1 1000
16 2 950
17 1 950
17 2 1000
18 1 950
18 2 950
19 1 1100
19 2 1050
20 1 1100
20 2 1050
21 1 1050
21 2 1100
22 1 1050
22 2 1050
23 1 1050
23 2 1100
24 1 1100
24 2 1100
25 1 1200
25 2 1150
26 1 1200
26 2 1200
27 1 1200
27 2 1200
28 1 1150
28 2 1200
29 1 1200
29 2 1150
30 1 1200
30 2 1150
31 1 1300
31 2 1300
32 1 1250
32 2 1300
33 1 1300
33 2 1300
34 1 1300
34 2 1250
35 1 1300
35 2 1250
36 1 1300
36 2 1250


In [589]:
'''
for k in K:
  for t in T:
    print('scene:',k,'time:',t,Q[k,t].X,'+',bin[k,t].X,'*',v[k,t-1].X,'+',1-bin[k,t].X,'*',u[k,t].X,'-',bin[k,t].X,'*',v[k,t].X,'=',x[k,t])
'''

"\nfor k in K:\n  for t in T:\n    print('scene:',k,'time:',t,Q[k,t].X,'+',bin[k,t].X,'*',v[k,t-1].X,'+',1-bin[k,t].X,'*',u[k,t].X,'-',bin[k,t].X,'*',v[k,t].X,'=',x[k,t])\n"

In [590]:
x[1,1]

750

In [591]:
for k in K:
  for t in T:
    print('scene:',k,'time:',t,Q[k,t].X,'+',v[k,t-1].X,'+',u[k,t].X,'-',v[k,t].X,'=',x[k,t])

scene: 1 time: 1 1700.0 + 0.0 + 0.0 - 950.0 = 750
scene: 1 time: 2 0.0 + 950.0 + 0.0 - 200.0 = 750
scene: 2 time: 1 1700.0 + 0.0 + 50.0 - 1000.0 = 750
scene: 2 time: 2 0.0 + 1000.0 + 0.0 - 200.0 = 800
scene: 3 time: 1 1700.0 + 0.0 + 0.0 - 950.0 = 750
scene: 3 time: 2 0.0 + 950.0 + 0.0 - 200.0 = 750
scene: 4 time: 1 1700.0 + 0.0 + 50.0 - 1000.0 = 750
scene: 4 time: 2 0.0 + 1000.0 + 0.0 - 200.0 = 800
scene: 5 time: 1 1700.0 + 0.0 + 50.0 - 950.0 = 800
scene: 5 time: 2 0.0 + 950.0 + -0.0 - 200.0 = 750
scene: 6 time: 1 1700.0 + 0.0 + 0.0 - 950.0 = 750
scene: 6 time: 2 0.0 + 950.0 + -0.0 - 200.0 = 750
scene: 7 time: 1 1700.0 + 0.0 + 50.0 - 900.0 = 850
scene: 7 time: 2 0.0 + 900.0 + 0.0 - 0.0 = 900
scene: 8 time: 1 1700.0 + 0.0 + 50.0 - 900.0 = 850
scene: 8 time: 2 0.0 + 900.0 + 0.0 - 0.0 = 900
scene: 9 time: 1 1700.0 + 0.0 + 0.0 - 850.0 = 850
scene: 9 time: 2 0.0 + 850.0 + 0.0 - 0.0 = 850
scene: 10 time: 1 1700.0 + 0.0 + 0.0 - 850.0 = 850
scene: 10 time: 2 0.0 + 850.0 + 0.0 - 0.0 = 850
scene